In [0]:
%run "../includes/configuration"

In [0]:
spark.read.json(f"{raw_folder_path}/2021-03-21/results.json").createOrReplaceTempView("results_cutover")

In [0]:
%sql
SELECT raceId, count(1)
FROM results_cutover
GROUP BY raceId
ORDER BY raceId DESC;

In [0]:
spark.read.json(f"{raw_folder_path}/2021-03-28/results.json").createOrReplaceTempView("results_w1")

In [0]:
%sql
SELECT raceId, count(1)
FROM results_w1
GROUP BY raceId
ORDER BY raceId DESC;

In [0]:
spark.read.json(f"{raw_folder_path}/2021-04-18/results.json").createOrReplaceTempView("results_w2")

In [0]:
%sql
SELECT raceId, count(1)
FROM results_w2
GROUP BY raceId
ORDER BY raceId DESC;

## Ingest results.json file
Step 1- Read the json file using the spark dataframe reader API

In [0]:
%run "../includes/configuration"

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, StructField, TimestampType, DateType, FloatType
from pyspark.sql.functions import col, struct,current_timestamp , concat, lit

In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", StringType(), True),
                                    StructField("statusId", StringType(), True)])

In [0]:
# Load the JSON file
results_df = spark.read\
    .schema(results_schema)\
    .json(f"{raw_folder_path}/{v_file_date}/results.json")

In [0]:
results_df= results_df.withColumn("fastestLapSpeed", col("fastestLapSpeed").cast(FloatType()))   

In [0]:
display(results_df)

In [0]:
results_df.printSchema()

#### Step 2 - Rename cloumns and add ingestion date

In [0]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id")\
                                    .withColumnRenamed("raceId", "race_id")\
                                    .withColumnRenamed("driverId", "driver_id")\
                                    .withColumnRenamed("constructorId", "constructor_id")\
                                    .withColumnRenamed("positionText", "position_text")\
                                    .withColumnRenamed("positionOrder", "position_order")\
                                    .withColumnRenamed("fastestLap", "fastest_lap")\
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time")\
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed")\
                                    .withColumn("data_source", lit(v_data_source))\
                                    .withColumn("file_date", lit(v_file_date))
                                    
                                           

In [0]:
results_with_columns_df = add_ingestion_date(results_with_columns_df)

In [0]:
display(results_with_columns_df)

#### Step 3 - Drop unwanted columns from the dataframe

In [0]:
results_final_df =results_with_columns_df.drop(col("statusId"))

de-dupe the dataframe

In [0]:
results_final_df = results_final_df.dropDuplicates(["driver_id", "race_id"])


#### Step 4 - Write to output to processed container in parquet format

method 1

In [0]:
# for race_id_list in results_final_df.select("race_id").distinct().collect():
#     if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
#      spark.sql(f"ALTER TABLE f1_processed.results DROP IF EXISTS PARTITION (race_id={race_id_list.race_id})")

In [0]:
##results_final_df.write.mode("overwrite").partitionBy("race_id").parquet(f"{processed_folder_path}/results")
# results_final_df.write.mode("append").partitionBy("race_id").format("parquet").saveAsTable("f1_processed.results")

Method 2

In [0]:
#overwrite_partition(results_final_df, "f1_processed", "results", "race_id")

In [0]:
merge_condition ="tgt.result_id =src.result_id AND tgt.race_id = src.race_id"
merge_delta_data (results_final_df,"f1_processed", "results",processed_folder_path,merge_condition,"race_id")

In [0]:
%sql
SELECT * FROM f1_processed.results

In [0]:
%sql
--DROP TABLE f1_processed.results;

In [0]:
%fs
ls /mnt/formula1dl2025practice/processed/results

In [0]:
#display(spark.read.parquet("/mnt/formula1dl2025practice/processed/results"))

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
SELECT race_id, 
COUNT(1)
 FROM f1_processed.results
 GROUP BY race_id
 ORDER BY race_id DESC;

In [0]:
%sql

SELECT max(race_year)
FROM f1_processed.results;